In [ ]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

In [ ]:
jogos = pd.read_csv('../data/processed/matches_FLA2024.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/processed/goal_scorers_2024.csv')
stats = pd.read_csv('../data/processed/stats_FLA2024.csv', parse_dates=['kick-off'])

### Plotando Gráficos

#### Número de Gols por mês

In [ ]:
new_order = ['January', 'February', 'March']#, 'April', 'May', 'June', 'July', 
             #'August', 'September', 'October', 'November', 'December']

gols_feitos_total = stats.groupby(stats['kick-off'].dt.month_name())['gols_marcados'].sum().reindex(new_order, axis=0)
gols_sofridos_total = stats.groupby(stats['kick-off'].dt.month_name())['gols_sofridos'].sum().reindex(new_order, axis=0)
jogos_mes = stats.groupby(stats['kick-off'].dt.month_name())['jogos_cumulativos'].count().reindex(new_order, axis=0)

saldo = gols_feitos_total - gols_sofridos_total

In [ ]:
labels = ['Janeiro', 'Fevereiro' 'Março']#, 'Abril', 'Maio', 'Junho', 'Julho', 
         # 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

fig, ax = plt.subplots(figsize=(20, 9))

cores = ['red' if saldo < 0 else 'blue' for saldo in saldo]

plt.title('Gols Marcados e Sofridos por Mês', fontsize=18)

plt.bar(saldo.index, saldo, align='center', label = 'Saldo de Gols', color=cores)

plt.plot(gols_feitos_total.index, gols_feitos_total, marker = 'o', c = 'green', label = 'Gols marcados')
plt.plot(gols_sofridos_total.index, gols_sofridos_total, marker = 'o', c = 'red', label = 'Gols sofridos')


for i in range(len(gols_feitos_total.index.to_list())):
    plt.text(gols_feitos_total.index.to_list()[i], gols_feitos_total.to_list()[i], 
             str(gols_feitos_total.to_list()[i]), color='white', ha='center', va='center', fontsize='13', 
             bbox=dict(facecolor='green', edgecolor='black'))
    plt.text(gols_feitos_total.index.to_list()[i], gols_sofridos_total.to_list()[i], 
             str(gols_sofridos_total.to_list()[i]), color='white', ha='center', va='center', fontsize='13',
             bbox=dict(facecolor='red', edgecolor='black'))
    plt.text(saldo.index.to_list()[i], saldo.to_list()[i], str(saldo.to_list()[i]), color='white', fontsize='13', 
             ha='center', va='center', bbox=dict(facecolor=cores[i], edgecolor='black'))
    


plt.ylim(-5, 25)
plt.ylabel('Gols', fontsize=16)
plt.xticks(gols_feitos_total.index, labels, fontsize=16)
plt.yticks(fontsize=16)

plt.legend(loc='upper right', fontsize=18)

plt.figtext(.91, 0.22, '@williamjouse', ha='right', va='top', alpha=0.5, fontsize=10, rotation=90)

plt.savefig('figures/figure.png', dpi = 300, bbox_inches='tight')
 
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 4))



axs[0].pie([vitorias, empates, derrotas], labels=['Vitórias', 'Empates', 'Derrotas'], 
           autopct='%1.1f%%', startangle=90, colors=['#27AE60', '#3498DB', '#E74C3C'], wedgeprops={'edgecolor': 'black'})

axs[0].set_title('Desempenho do Flamengo no ano')

labels = ['Jogos', 'Vitórias', 'Empates', 'Derrotas', 'Gols Pró', 'Gols Contra']
values = [total_jogos, vitorias, empates, derrotas, gols_pro, gols_contra]
         
axs[1].barh(labels, values, color=['#27AE60', 'black', '#E74C3C', '#3498DB'], edgecolor='black')

for i, value in enumerate(values):
    axs[1].text(value + 1, i, str(value), va='center')
    
import seaborn as sns    
sns.barplot(x=values, y=labels, ax=axs[1], palette='viridis')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
axs[1].spines['left'].set_visible(False)
axs[1].set_xticks([])


axs[1].set_title('Estatísticas Gerais')


plt.subplots_adjust(wspace=0.4)

plt.figtext(1.0, 0.23, '@williamjouse', ha='right', va='top', alpha=0.5, fontsize=10, rotation=90)

plt.savefig('figures/figure2.png', dpi = 300, bbox_inches='tight')


plt.show()

In [ ]:
jogos['Mês'] = jogos['kick-off'].dt.month_name()

agrupado = jogos.groupby(['Mês', 'resultado']).size().unstack().reindex(new_order, axis=0)


vic = agrupado['Vitória'].fillna(0).astype('Int64').to_list()
emp = agrupado['Empate'].fillna(0).astype('Int64').to_list()
der = agrupado['Derrota'].fillna(0).astype('Int64').to_list()
months = agrupado.index.to_list()

In [ ]:
meses = ['Janeiro', 'Fevereiro', 'Março']#, 'Abril', 'Maio', 'Junho', 
         #'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

x = range(len(months))
largura_barra = 0.2

fig, ax = plt.subplots(figsize=(20, 9))

plt.title('Desempenho por Mês', fontsize=18)

plt.bar(x, vic, width=largura_barra, label='Vitórias', color='green')
plt.bar([i + largura_barra for i in x], emp, width=largura_barra, label='Empates', color='blue')
plt.bar([i + 2 * largura_barra for i in x], der, width=largura_barra, label='Derrotas', color='red')


for i, v in enumerate(vic):
    plt.text(i, v, str(v), ha='center', va='bottom', color='white', fontweight='bold', fontsize='13',
             bbox=dict(facecolor='green', edgecolor='none', pad=2))
for i, e in enumerate(emp):
    plt.text(i + largura_barra, e, str(e), ha='center', va='bottom', color='white', fontweight='bold', fontsize='13',
             bbox=dict(facecolor='blue', edgecolor='none', pad=2))
for i, d in enumerate(der):
    plt.text(i + 2 * largura_barra, d, str(d), ha='center', va='bottom', color='white', fontweight='bold', fontsize='13',
             bbox=dict(facecolor='red', edgecolor='none', pad=2))

plt.xticks([i + largura_barra for i in x], meses, fontsize=16)

plt.yticks(fontsize=18)
plt.ylim(0, max(vic + emp + der) + 2)

# Adicionando uma legenda
green_patch = mpatches.Patch(color='green', label='Vitórias')
blue_patch = mpatches.Patch(color='blue', label='Empates')
red_patch = mpatches.Patch(color='red', label='Derrotas')
plt.legend(handles=[green_patch, blue_patch, red_patch], fontsize=16)
#plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=16)

plt.figtext(.91, 0.22, '@williamjouse', ha='right', va='top', alpha=0.5, fontsize=10, rotation=90)

plt.legend(loc='upper right', fontsize=16)

plt.savefig('figures/figure3.png', dpi = 300, bbox_inches='tight')

plt.show()